# Generative Happy Moments Using Char-RNN

Char-RNN implements multi-layer Recurrent Neural Network (RNN, LSTM, and GRU) for training/sampling from character-level language models. In other words the model takes one text file as input and trains a Recurrent Neural Network that learns to predict the next character in a sequence. The RNN can then be used to generate text character by character that will look like the original training data. This network is first posted by Andrej Karpathy, you can find out about his original code on https://github.com/karpathy/char-rnn, the original code is written in *lua*.

In [18]:
import time
import numpy as np
import tensorflow as tf
import pandas as pd
import re
# Notebook auto reloads code. (Ref: http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Setup
In this part, we will use only the cleaned hm data

In [3]:
data = pd.read_csv("../data/cleaned_hm.csv")
data.head()

,hmid,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
0,27673,2053,24h,I went on a successful date with someone I fel...,I went on a successful date with someone I fel...,True,1,NaN,affection
1,27674,2,24h,I was happy when my son got 90% marks in his e...,I was happy when my son got 90% marks in his e...,True,1,NaN,affection
2,27675,1936,24h,I went to the gym this morning and did yoga.,I went to the gym this morning and did yoga.,True,1,NaN,exercise
3,27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
4,27677,6227,24h,I went with grandchildren to butterfly display...,I went with grandchildren to butterfly display...,True,1,NaN,affection


In [7]:
text = " ".join([words for words in data["cleaned_hm"]])
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))
# and let's get a glance of what the text is
print(text[:500])

Length of text: 9460832 characters
I went on a successful date with someone I felt sympathy and connection with. I was happy when my son got 90% marks in his examination  I went to the gym this morning and did yoga. We had a serious talk with some friends of ours who have been flaky lately. They understood and we had a good evening hanging out. I went with grandchildren to butterfly display at Crohn Conservatory
 I meditated last night. I made a new recipe for peasant bread, and it came out spectacular! I got gift from my elder 


This is to strip all the weird extra characters

In [8]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

96 unique characters


In [9]:
# Creating a mapping from unique characters to indices
vocab_to_ind = {c: i for i, c in enumerate(vocab)}
ind_to_vocab = dict(enumerate(vocab))
text_as_int = np.array([vocab_to_ind[c] for c in text], dtype=np.int32)

# We mapped the character as indexes from 0 to len(vocab)
for char,_ in zip(vocab_to_ind, range(20)):
    print('{:6s} ---> {:4d}'.format(repr(char), vocab_to_ind[char]))
# Show how the first 10 characters from the text are mapped to integers
print ('{} --- characters mapped to int --- > {}'.format(text[:10], text_as_int[:10]))

'\t'   --->    0
'\n'   --->    1
'\r'   --->    2
' '    --->    3
'!'    --->    4
'"'    --->    5
'#'    --->    6
'$'    --->    7
'%'    --->    8
'&'    --->    9
"'"    --->   10
'('    --->   11
')'    --->   12
'*'    --->   13
'+'    --->   14
','    --->   15
'-'    --->   16
'.'    --->   17
'/'    --->   18
'0'    --->   19
I went on  --- characters mapped to int --- > [44  3 89 71 80 86  3 81 80  3]


## Creating Batches Using Generators
We're going to partition our data

1. **batch_size**: Reviewing batches in CNN, if we have 100 samples and we set batch_size as 10, it means that we will send 10 samples to the network at one time. In RNN, batch_size have the same meaning, it defines how many samples we send to the network at one time.
2. **sequence_length**: However, as for RNN, we store memory in our cells, we pass the information through cells, so we have this sequence_length concept, which also called 'steps', it defines how long a sequence is.

From above two concepts, we here clarify the meaning of batch_size in RNN. Here, we define the number of sequences in a batch as N and the length of each sequence as M, so batch_size in RNN **still** represent the number of sequences in a batch but the data size of a batch is actually an array of size **[N, M]**.

In [10]:
def get_batches(array, n_seqs, n_steps):
    '''
    Partition data array into mini-batches
    input:
    array: input data
    n_seqs: number of sequences in a batch N
    n_steps: length of each sequence  M
    output:
    x: inputs
    y: targets, which is x with one position shift
       you can check the following figure to get the sence of what a target looks like
    '''
    batch_size = n_seqs * n_steps
    n_batches = int(len(array) / batch_size)
    batch_count=0
    # we only keep the full batches and ignore the left.
    x = array[:batch_size * n_batches]
    y = array[1:batch_size * n_batches +1]
    x = x.reshape((n_seqs, -1))
    y = y.reshape((n_seqs, -1))
    # You should now create a loop to generate batches for inputs and targets

    while True:
        if batch_count < n_batches:
            yield (x[:,batch_count * n_steps : (batch_count +1) * n_steps], 
                   y[:,batch_count * n_steps : (batch_count +1) * n_steps]) # yield new batch
            batch_count += 1 # update counter
        else:
            batch_count = 0 # reset counter when we went through all data
        
        

Sanity check

In [11]:
batches= get_batches(text_as_int, 10, 10)
x, y = next(batches)
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[44  3 89 71 80 86  3 81 80  3]
 [67 88 81 84 75 86 71  3 85 74]
 [ 3 67  3 80 75 69 71  3 67 80]
 [ 4  3 75  3 73 81  3 86 81  3]
 [ 3 67 86  3 67  3 69 81 79 82]
 [67 86 71 84  3 85 74 71  3 89]
 [74  3 51 85 91 69 74  3 81 80]
 [71  3 74 75 79  3 67  3 85 86]
 [67 73 71 17  3 55 74 71  3 79]
 [86 74  3 79 91  3 68 71 85 86]]

y
 [[ 3 89 71 80 86  3 81 80  3 67]
 [88 81 84 75 86 71  3 85 74 81]
 [67  3 80 75 69 71  3 67 80 70]
 [ 3 75  3 73 81  3 86 81  3 81]
 [67 86  3 67  3 69 81 79 82 67]
 [86 71 84  3 85 74 71  3 89 81]
 [ 3 51 85 91 69 74  3 81 80  3]
 [ 3 74 75 79  3 67  3 85 86 71]
 [73 71 17  3 55 74 71  3 79 67]
 [74  3 79 91  3 68 71 85 86  3]]


In [12]:
x.shape , y.shape

((10, 10), (10, 10))

## Build Char-RNN model
We will build our char-rnn model, it consists of input layer, rnn_cell layer, output layer, loss and optimizer, we will build them one by one.

The goal is to predict new text after given prime word, so for our training data, we have to define inputs and targets.

Importing CharRNN class files. Contains both LSTM and GRU cells

In [15]:
from CharRNN import *

### Training
Set sampling as False(default), we can start training the network, we automatically save checkpoints in the folder /checkpoints.

In [16]:
# these are preset parameters, you can change them to get better result
batch_size = 100         # Sequences per batch
num_steps = 100          # Number of sequence steps per batch
rnn_size = 256           # Size of hidden layers in rnn_cell
num_layers = 2           # Number of hidden layers
learning_rate = 0.005    # Learning rate
keep_prob=0.5
cell_type='LSTM'
grad_clip=5
train_keep_prob=0.5
max_count=3000
save_every_n=500

In [17]:
model = CharRNN(num_classes=len(vocab),
                batch_size=batch_size,
                num_steps=num_steps,
                cell_type=cell_type, 
                rnn_size=rnn_size,
                num_layers=num_layers,
                learning_rate=learning_rate,
                grad_clip=grad_clip,
                train_keep_prob=train_keep_prob,
                sampling=False)
batches = get_batches(text_as_int, batch_size, num_steps)
model.train(batches, max_count, save_every_n)

step: 200  loss: 2.0953  0.2262 sec/batch
step: 400  loss: 1.7041  0.2213 sec/batch
step: 600  loss: 1.5366  0.2163 sec/batch
step: 800  loss: 1.4672  0.2208 sec/batch
step: 1000  loss: 1.4641  0.2203 sec/batch
step: 1200  loss: 1.3372  0.2177 sec/batch
step: 1400  loss: 1.3356  0.2187 sec/batch
step: 1600  loss: 1.3520  0.2253 sec/batch
step: 1800  loss: 1.3085  0.2211 sec/batch
step: 2000  loss: 1.3172  0.2220 sec/batch
step: 2200  loss: 1.2716  0.2161 sec/batch
step: 2400  loss: 1.2783  0.2181 sec/batch
step: 2600  loss: 1.2904  0.2214 sec/batch
step: 2800  loss: 1.2654  0.2211 sec/batch
step: 3000  loss: 1.2168  0.2184 sec/batch
step: 3200  loss: 1.2611  0.2225 sec/batch
step: 3400  loss: 1.2426  0.2172 sec/batch
step: 3600  loss: 1.2458  0.2188 sec/batch
step: 3800  loss: 1.2176  0.2215 sec/batch
step: 4000  loss: 1.2240  0.2254 sec/batch
step: 4200  loss: 1.2091  0.2237 sec/batch
step: 4400  loss: 1.1978  0.2236 sec/batch
step: 4600  loss: 1.2081  0.2197 sec/batch
step: 4800  los

In [19]:
# look up checkpoints
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i6000_l256_LSTM.ckpt"
all_model_checkpoint_paths: "checkpoints/i4000_l256_LSTM.ckpt"
all_model_checkpoint_paths: "checkpoints/i4500_l256_LSTM.ckpt"
all_model_checkpoint_paths: "checkpoints/i5000_l256_LSTM.ckpt"
all_model_checkpoint_paths: "checkpoints/i5500_l256_LSTM.ckpt"
all_model_checkpoint_paths: "checkpoints/i6000_l256_LSTM.ckpt"

### Let's Make Some Samples

In [25]:
model = CharRNN(num_classes=len(vocab),
                batch_size=batch_size,
                num_steps=num_steps,
                cell_type=cell_type, 
                rnn_size=rnn_size,
                num_layers=num_layers,
                learning_rate=learning_rate,
                grad_clip=grad_clip,
                train_keep_prob=train_keep_prob,
                sampling=True)
# choose the last checkpoint and generate new text
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = model.sample(checkpoint, 1000, len(vocab), vocab_to_ind, ind_to_vocab, prime="My ")
print(samp)


INFO:tensorflow:Restoring parameters from checkpoints/i6000_l256_LSTM.ckpt
My single day. My friends chicks as a trial and it starced from house, and I cound on a colference than with me and she was how and-well.  That I was supporming well for a nice milifor. I was very happy to be molling. My has so imprevie and she thought it's a feeling in along one at work and I went to my food wanting. I am selected in a few months of music, and she had been complemented about her studyed while I was a hit to see the feeds the words its satisvince in a car off. I went to bed after several open those arcoming spishes for some founday i got to gring a cell conventes of that dog which I got some tried tirlergen ticlenes after shrold and the break. I had a good time with my best friond we are able to get to be a long. I had the place. At my friend to work than any and this west man terrowing astererning to have a good discussion of a stress as the mark i asked me. I gave it to going to a present in m

It definitely learned the structure. The 1000th iteration was much more gibberish

In [24]:
samp = model.sample("checkpoints/i6000_l256_LSTM.ckpt", 1000, len(vocab), vocab_to_ind, ind_to_vocab, prime="I'm ")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i6000_l256_LSTM.ckpt
I'm has a tempore.
 My friends and I mnt taught until that ware also selecting was a lot of free of thinks to management that I wanted in a few days!.. I goita with my friends to see my glan app on a chance to that was able to see it friends.  My friend gett our big project as well and they were happy is a standing bestie stains and I was abreaved where it was a high couch and so we was able to finish my car and I have toding twe we we sundanged, still are a few friends from my boyfriend and money with my favorite back and I hadn't hit my home and I saw me a log that I was. We ate a boss and shipent that I wanted.
This morning my be a boour. I had seag with a new place of happy mother and hang out to the food was this the a macriay, I headed troughing out. My sister steak a surprise at that time.  My stare won my both and hot a task wish show.'I started from a concept and so that I wanted for months. This mornin